### Computing climatology, testing settings for Contrack

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from matplotlib import cm

In [2]:
##some data
##EERIE
id_exp='ifs-fesom2-sr' #IFS_9-FESOM_5-production, IFS_9-FESOM_5-production-hist
xr_in=xr.open_mfdataset('/scratch/b/b382006/EERIE/cycle1/'+id_exp+'/z500/3D_6hourly/z500_ifs_c1_19*')

In [3]:
### take a sample
xr_in=xr_in.sel(time=slice('1970','1999'))
xr_in = xr_in.resample(time='D').mean()
xr_in

<xarray.Dataset> Size: 6GB
Dimensions:  (time: 10957, lat: 181, lon: 360)
Coordinates:
  * lat      (lat) int64 1kB -90 -89 -88 -87 -86 -85 -84 ... 85 86 87 88 89 90
  * lon      (lon) int64 3kB -180 -179 -178 -177 -176 ... 175 176 177 178 179
  * time     (time) datetime64[ns] 88kB 1970-01-01 1970-01-02 ... 1999-12-31
Data variables:
    z        (time, lat, lon) float64 6GB dask.array<chunksize=(1, 181, 360), meta=np.ndarray>

In [12]:
# Define rolling window for climatology (8 years for long-term trends)
smooth_window = 8
clim = xr_in['z'].rolling(time=smooth_window, center=True).mean().fillna(xr_in['z'].isel(time=slice(0, smooth_window)).mean('time'))
clim

<xarray.DataArray 'z' (time: 10957, lat: 181, lon: 360)> Size: 6GB
dask.array<where, shape=(10957, 181, 360), dtype=float64, chunksize=(8, 19, 33), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) int64 1kB -90 -89 -88 -87 -86 -85 -84 ... 85 86 87 88 89 90
  * lon      (lon) int64 3kB -180 -179 -178 -177 -176 ... 175 176 177 178 179
  * time     (time) datetime64[ns] 88kB 1970-01-01 1970-01-02 ... 1999-12-31
Attributes: (12/28)
    paramId:                             129
    dataType:                            fc
    numberOfPoints:                      1038240
    typeOfLevel:                         isobaricInhPa
    stepUnits:                           1
    stepType:                            instant
    ...                                  ...
    latitudeOfFirstGridPointInDegrees:   90.0
    longitudeOfFirstGridPointInDegrees:  0.0
    iDirectionIncrementInDegrees:        0.25
    jDirectionIncrementInDegrees:        0.25
    iScansNegatively:                    0
    jScansPositively:                    0

In [ ]:
# Remove seasonality (31-day window)
seasonal_window = 31
groupby = "dayofyear"
clim2 = clim.groupby("time.dayofyear").mean("time")
clim2 = clim2.rolling(**{groupby: seasonal_window}, center=True).mean().fillna(clim2[-seasonal_window:].mean(dim=groupby))

# Compute anomaly by subtracting climatology
anom = (xr_in['z'].groupby('time.dayofyear') - clim2)

anom

In [16]:
###

In [ ]:
### File from EERIE with no data
#test=xr.open_dataset('/scratch/b/b382006/EERIE/cycle1/'+id_exp+'/z500/3D_6hourly/z500_ifs_c1_2008_day.nc')

In [18]:
#test.z.plot()